In [1]:
#%pip install -q tensorflow-recommenders
import tensorflow as tf
import pandas as pd
from tensorflow.keras.layers import Embedding, Dense, StringLookup
from tensorflow.keras import Sequential
import tensorflow_recommenders as tfrs
from tensorflow.keras.metrics import AUC
import math
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import keras
from keras.src.layers import LSTM

In [2]:
if tf.config.list_physical_devices('GPU'):
    print("GPU is available")
else:
    print("GPU is not available")

GPU is available


In [3]:
# Load data
basepath = '../../datasets/preprocessed_datasets/gabor/'
pandas_interaction_data = pd.read_pickle(basepath + 'user_item_interactions_only_main_products.pkl')
pandas_interaction_data = pandas_interaction_data.sample(9000)
pandas_interaction_data.to_pickle(basepath + 'user_item_interactions_only_main_products_sample_10k-2.pkl')
pandas_interaction_data.amount = pandas_interaction_data.amount.apply(lambda x: 1 if x > 0 else 0) 
product_data = pd.read_pickle(basepath + 'no_product_variants_with_duplicate_ids.pkl')
pandas_interaction_data.main_product_id = pandas_interaction_data.main_product_id.astype(int)
product_data.main_product_id = product_data.main_product_id.astype(int)
product_data['color_mapped'] = pd.factorize(product_data['productColorName'])[0]

pandas_interaction_data = pd.merge(pandas_interaction_data, product_data[['main_product_id', 'color_mapped', 'productName']],
                                   left_on='main_product_id',
                                   right_on='main_product_id',
                                    how='left')

display(pandas_interaction_data, product_data)

,customer_id,main_product_id,amount,color_mapped,productName
0,8972709,3696847,1,2,Sneaker low Glattleder weiß
1,5698916,4775789,1,8,Spangenpumps Rauleder rosa
2,9473882,5096560,1,2,Plateau Sandale Glattleder weiß
3,6753736,3505964,1,4,Sneaker low Materialmix Leder/Lederimitat grau
4,5812686,549920,1,0,elegante Ballerinas Lackleder schwarz
...,...,...,...,...,...
8995,2017525,4125038,1,6,Schnürstiefelette Glattleder beige
8996,3915170,3505101,1,3,Sneaker low Rauleder braun
8997,9907290,3591570,1,3,Sneaker low Materialmix Leder braun
8998,8275774,3711307,1,0,Sneaker low Glattleder schwarz


,main_product_id,productNumber,productName,productColorName,articleNumber,brand__id,mainCategory__id,gender,originCountry,line,...,sizeEu,sizeUk,sizeIndex,shaftLength__value,sole,isTransferee,isSuccessor,duplicate_product__ids,product_sizes,color_mapped
0,557559,4058394021466,sportliche Ballerinas Glattleder schwarz,schwarz,02.643.57,6590678,315571,w,PT,F-S|H-W,...,40.0,6.5,11.0,0.0,Gummi,0.0,0.0,"[557559, 549733, 549734, 557553, 589488, 58948...","[35.5, 36.0, 37.0, 37.5, 38.0, 38.5, 39.0, 40....",0
1,551622,4054452768212,elegante Pumps Glattleder schwarz,schwarz,05.160.37,6590677,315573,w,PT,F-S|H-W,...,38.0,5.0,8.0,0.0,EVA,0.0,0.0,"[551622, 551505, 551510, 551509, 552973, 55162...","[35.0, 35.5, 36.0, 37.0, 37.5, 38.0, 38.5, 39....",0
2,547193,4054452768427,Slipper Glattleder schwarz,schwarz,04.443.27,6590677,315576,w,SK,F-S|H-W,...,42.0,8.0,14.0,0.0,PU-TPU,0.0,0.0,"[547193, 547191, 547196, 547187, 547189, 54719...","[35.0, 35.5, 36.0, 37.0, 37.5, 38.0, 38.5, 39....",0
3,549776,4059701687894,Sneaker low Rauleder blau,blau,06.968.46,4997827,315567,w,VN,F-S|H-W,...,39.0,6.0,10.0,0.0,Gummi-EVA,0.0,0.0,"[549776, 550599, 550596, 550593, 550601, 54978...","[35.0, 35.5, 36.0, 37.0, 37.5, 38.0, 38.5, 39....",1
4,550726,4054452851594,elegante Ballerinas Materialmix Lederimitat sc...,schwarz,06.102.67,6590678,315574,w,SK,F-S|H-W,...,37.5,4.5,7.0,0.0,EVA,0.0,0.0,"[550726, 550736, 550735, 550732, 550728, 55072...","[35.0, 37.0, 37.5, 38.0, 38.5, 39.0, 40.0, 40....",0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3292,10603553,4066558951733,Sandale met plateauzool Suède blauw,blauw,24.764.36,6590677,315568,w,SK,F-S,...,35.0,2.5,3.0,0.0,PU-TPU,0.0,0.0,"[10603553, 10544662]","[35.0, 37.0]",30
3293,10442948,4065171827272,Mokassin Materialmix Leder pink,pink,26.090.21,6590678,315590,w,PT,F-S,...,40.5,7.0,12.0,0.0,Gummi,0.0,0.0,"[10442948, 10442949]","[40.5, 41.0]",56
3294,10782704,4251234499207,Shopper ANDIE blau,blau,921453,363013,363017,w,DE,F-S,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[10782704],[],1
3295,10679703,4066558303617,Elegante pumps Glad leer wit,wit,21.450.60,6590677,315573,w,PT,F-S,...,35.0,2.5,3.0,0.0,TPU,0.0,0.0,[10679703],[35.0],33


In [4]:
# Initialize tokenizer
string_lookup = StringLookup()
string_lookup.adapt(pandas_interaction_data['productName'])

tokenizer = Tokenizer(num_words=string_lookup.vocabulary_size(), oov_token="<OOV>")
tokenizer.fit_on_texts(pandas_interaction_data['productName'])

# Convert text to sequences
sequences = tokenizer.texts_to_sequences(pandas_interaction_data['productName'])

# Pad sequences
padded_sequences = pad_sequences(sequences, padding='post')

In [5]:
# Create a tf.data.Dataset from the interaction data
interaction_dataset = tf.data.Dataset.from_tensor_slices({
    'customer_id': pandas_interaction_data['customer_id'].values,
    'main_product_id': pandas_interaction_data['main_product_id'].values,
    'amount': pandas_interaction_data['amount'].values, 
    'product_color': pandas_interaction_data['color_mapped'].values, 
    'product_description': padded_sequences, 
})
#variables 
dataset_len = pandas_interaction_data.shape[0]
test_len = math.ceil(dataset_len * 0.2)
train_len = dataset_len - test_len

metrics_batchsize = 16
train_batch_size = 128
test_batch_size = 64
random_seed = 27

In [6]:
interaction_dataset

<_TensorSliceDataset element_spec={'customer_id': TensorSpec(shape=(), dtype=tf.int64, name=None), 'main_product_id': TensorSpec(shape=(), dtype=tf.int64, name=None), 'amount': TensorSpec(shape=(), dtype=tf.int64, name=None), 'product_color': TensorSpec(shape=(), dtype=tf.int64, name=None), 'product_description': TensorSpec(shape=(7,), dtype=tf.int32, name=None)}>

In [7]:
#train test split
tf.random.set_seed(random_seed)
# shuffled = interaction_dataset.shuffle(len, seed=random_seed, reshuffle_each_iteration=False)
# train = shuffled.take(train_len)
# test = shuffled.skip(train_len).take(test_len)
shuffled = interaction_dataset.shuffle(dataset_len, seed=random_seed, reshuffle_each_iteration=False)

train = shuffled.take(train_len)
test = shuffled.skip(train_len).take(test_len)
display(train, test)

<_TakeDataset element_spec={'customer_id': TensorSpec(shape=(), dtype=tf.int64, name=None), 'main_product_id': TensorSpec(shape=(), dtype=tf.int64, name=None), 'amount': TensorSpec(shape=(), dtype=tf.int64, name=None), 'product_color': TensorSpec(shape=(), dtype=tf.int64, name=None), 'product_description': TensorSpec(shape=(7,), dtype=tf.int32, name=None)}>

<_TakeDataset element_spec={'customer_id': TensorSpec(shape=(), dtype=tf.int64, name=None), 'main_product_id': TensorSpec(shape=(), dtype=tf.int64, name=None), 'amount': TensorSpec(shape=(), dtype=tf.int64, name=None), 'product_color': TensorSpec(shape=(), dtype=tf.int64, name=None), 'product_description': TensorSpec(shape=(7,), dtype=tf.int32, name=None)}>

In [8]:
# Unique customer and product identifiers
unique_customer_ids = np.array(pandas_interaction_data["customer_id"].unique())
unique_product_ids = np.array(pandas_interaction_data["main_product_id"].unique())

unique_colors = np.array(product_data["color_mapped"].unique())
user_ids_vocabulary = tf.keras.layers.IntegerLookup(mask_token=None)
user_ids_vocabulary.adapt(unique_customer_ids)

product_ids_vocabulary = tf.keras.layers.IntegerLookup(mask_token=None)
product_ids_vocabulary.adapt(unique_product_ids)

product_colors_vocabulary = tf.keras.layers.IntegerLookup(mask_token=None)
product_colors_vocabulary.adapt(unique_colors)
# print(unique_product_ids.shape[0], unique_customer_ids.shape[0], unique_product_ids, unique_product_ids)

In [9]:
# Define a model using TensorFlow Recommenders
product_ids_dataset = tf.data.Dataset.from_tensor_slices(unique_product_ids)
class RetailModel(tfrs.Model):

    def __init__(self, unique_product_ids_vocab, unique_customer_ids_vocab, unique_product_color_vocab, embedding_dimension=32):
        super().__init__()
        # Set up user and product representations
        self.user_embedding = tf.keras.Sequential([
            unique_customer_ids_vocab,
            Embedding(unique_customer_ids_vocab.vocabulary_size(), embedding_dimension)
        ])
        self.product_embedding = tf.keras.Sequential([
            unique_product_ids_vocab,
            Embedding(unique_product_ids_vocab.vocabulary_size(), embedding_dimension)
        ])

        # Color embeddings
        self.color_embedding = tf.keras.Sequential([
            unique_product_color_vocab,
            Embedding(unique_product_color_vocab.vocabulary_size(), embedding_dimension)
        ]) 


        # Textual description model
        self.textual_description_model = tf.keras.Sequential([
            Embedding(input_dim=unique_product_ids_vocab.vocabulary_size(), output_dim=embedding_dimension),
            LSTM(64),
            Dense(64, activation='relu')
        ])

        # New embeddings for color and description
        self.reduced_product_embeddings = Dense(64, activation='relu')
        self.reduced_description_embeddings = Dense(32, activation='relu')
        self.reduced_color_embeddings = Dense(32, activation='relu')
        
        # Set up a dense layer for the task.
        self.dense_layer = Dense(128, input_shape=(256,), activation="relu")

        # Set up retrieval task and metrics
        self.task = tfrs.tasks.Retrieval(
            metrics=tfrs.metrics.FactorizedTopK(candidates=product_ids_dataset.batch(metrics_batchsize).map(self.product_model))
        )
        self.auc_metric = AUC(name='auc')
        self.rmse_metric = tf.keras.metrics.RootMeanSquaredError(name='rmse')
        self.precision = tf.keras.metrics.Precision(name='precision')
        self.recall = tf.keras.metrics.Recall(name='recall')

    def product_model(self, product_ids):
        return self.product_embedding(product_ids)

    def dot_product_score(self, user, product):
        """
        Computes the dot product between user and product embeddings to get the interaction score.
        """
        return tf.reduce_sum(user * product, axis=1)

    def compute_loss(self, features, training=False):
        # print(features)
        user_embeddings = self.user_embedding(features["customer_id"])
        product_embeddings = self.product_embedding(features["main_product_id"])

        color_embeddings = self.color_embedding(features["product_color"])
        description_embeddings = self.textual_description_model(features["product_description"])

        reduced_product_embeddings = self.reduced_product_embeddings(product_embeddings)
        reduced_description_embeddings = self.reduced_description_embeddings(description_embeddings)
        reduced_color_embeddings = self.reduced_color_embeddings(color_embeddings)


        # Now concatenate
        user_combined = tf.concat([user_embeddings, ], axis=1)  # shape will be [None, 128]
        product_combined = tf.concat([reduced_product_embeddings, reduced_color_embeddings, reduced_description_embeddings], axis=1)  # shape will be [None, 128]
        


        user_output = self.dense_layer(user_embeddings)
        product_output = self.dense_layer(product_combined)
        # This is a hypothetical function that returns the logits or scores
        # This needs to be adapted based on your actual model
        positive_logits = self.dot_product_score(user_output, product_output)

        # Update RMSE
        self.rmse_metric.update_state(y_true=features["amount"], y_pred=positive_logits)


        return self.task(user_output, product_output)

    def evaluate(self, validation_dataset, *args, **kwargs):
        # Call the base class's evaluate method
        
    
        # Compute additional metrics
        for features in validation_dataset:
            user_embeddings = self.user_embedding(features["customer_id"])
            product_embeddings = self.product_embedding(features["main_product_id"])
            
            user_output = self.dense_layer(user_embeddings)
            product_output = self.dense_layer(product_embeddings)
            
            positive_logits = self.dot_product_score(user_output, product_output)
            # Assuming you have a binary "label" in your dataset indicating 1 for positive interaction and 0 for negative
            self.precision.update_state(y_true=features["amount"], y_pred=positive_logits)
            self.recall.update_state(y_true=features["amount"], y_pred=positive_logits)
    
        precision_result = self.precision.result().numpy()
        recall_result = self.recall.result().numpy()
    
        # Calculate F1 score
        if (precision_result + recall_result) != 0:
            f1_score = 2 * (precision_result * recall_result) / (precision_result + recall_result)
        else:
            f1_score = 0.0
    
        # Reset the metrics for the next evaluation
        # self.precision.reset_states()
        # self.recall.reset_states()
        base_results = super(RetailModel, self).evaluate(validation_dataset, *args, **kwargs)
    
        return base_results, [precision_result, recall_result, f1_score]  # or append additional results as needed



In [10]:
model = RetailModel(product_ids_vocabulary, user_ids_vocabulary, product_colors_vocabulary, embedding_dimension=128)
model.compile(optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.001)) # using legacy instead of tf.keras.optimizers.Adagrad, because newer version is slow on m1/m2 macs

In [11]:
# Shuffle, batch, and cache the data.
cached_train = train.shuffle(dataset_len).batch(train_batch_size).cache()
cached_test = test.batch(test_batch_size).cache()
# Train the model
# model.fit(cached_train, validation_data=cached_test, validation_freq=5, epochs=3) 
model.fit(cached_train, validation_freq=5, epochs=3)

Epoch 1/3
57/57 [==============================] - 26s 428ms/step - auc: 0.0000e+00 - rmse: 0.9916 - precision: 0.0000e+00 - recall: 0.0000e+00 - factorized_top_k/top_1_categorical_accuracy: 0.1181 - factorized_top_k/top_5_categorical_accuracy: 0.1821 - factorized_top_k/top_10_categorical_accuracy: 0.2165 - factorized_top_k/top_50_categorical_accuracy: 0.3200 - factorized_top_k/top_100_categorical_accuracy: 0.3810 - loss: 603.4828 - regularization_loss: 0.0000e+00 - total_loss: 603.4828
Epoch 2/3
57/57 [==============================] - 28s 486ms/step - auc: 0.0000e+00 - rmse: 1.8896 - precision: 0.0000e+00 - recall: 0.0000e+00 - factorized_top_k/top_1_categorical_accuracy: 0.9464 - factorized_top_k/top_5_categorical_accuracy: 0.9638 - factorized_top_k/top_10_categorical_accuracy: 0.9694 - factorized_top_k/top_50_categorical_accuracy: 0.9797 - factorized_top_k/top_100_categorical_accuracy: 0.9849 - loss: 582.4715 - regularization_loss: 0.0000e+00 - total_loss: 582.4715
Epoch 3/3
57/57 

In [12]:
# Evaluate the model
k = 5
result_evaluate_train = model.evaluate(cached_train)
result_evaluate_train

57/57 [==============================] - 29s 503ms/step - auc: 0.0000e+00 - rmse: 28.4430 - precision: 0.0000e+00 - recall: 0.0000e+00 - factorized_top_k/top_1_categorical_accuracy: 1.0000 - factorized_top_k/top_5_categorical_accuracy: 1.0000 - factorized_top_k/top_10_categorical_accuracy: 1.0000 - factorized_top_k/top_50_categorical_accuracy: 1.0000 - factorized_top_k/top_100_categorical_accuracy: 1.0000 - loss: 242.1365 - regularization_loss: 0.0000e+00 - total_loss: 242.1365


([0.0,
  28.44295883178711,
  0.0,
  0.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  29.919248580932617,
  0,
  29.919248580932617],
 [1.0, 0.89652777, 0.9454412244264775])

In [13]:
result_evaluat_test = model.evaluate(cached_test)
result_evaluat_test

29/29 [==============================] - 16s 560ms/step - auc: 0.0000e+00 - rmse: 18.4325 - precision: 0.0000e+00 - recall: 0.0000e+00 - factorized_top_k/top_1_categorical_accuracy: 1.0000 - factorized_top_k/top_5_categorical_accuracy: 1.0000 - factorized_top_k/top_10_categorical_accuracy: 1.0000 - factorized_top_k/top_50_categorical_accuracy: 1.0000 - factorized_top_k/top_100_categorical_accuracy: 1.0000 - loss: 363.0824 - regularization_loss: 0.0000e+00 - total_loss: 363.0824


([0.0,
  18.432540893554688,
  0.0,
  0.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  44.24104309082031,
  0,
  44.24104309082031],
 [1.0, 0.91944444, 0.9580318364780054])

In [14]:
print("Name\t\tValue")
print("-" * 30)
for metric in model.metrics:
    print(f"{metric.name}\t\t{metric.result().numpy()}")
print(f"AUC: {model.auc_metric.result().numpy()}")

Name		Value
------------------------------
auc		0.0
rmse		18.432540893554688
precision		0.0
recall		0.0
factorized_top_k/top_1_categorical_accuracy		1.0
factorized_top_k/top_5_categorical_accuracy		1.0
factorized_top_k/top_10_categorical_accuracy		1.0
factorized_top_k/top_50_categorical_accuracy		1.0
factorized_top_k/top_100_categorical_accuracy		1.0
AUC: 0.0


In [15]:
sample = next(iter(cached_test.take(1)))

# Extract customer_id from the sample
customer_id = sample['customer_id'].numpy()
customer_id

array([ 8494801,  9726654,  9448368,  5898138,  2075624,  9802315,
        7995279,  2179131,  7869607,  9563631,  3492130,  5872093,
       10074844,  8895916,  8814730,  7107466,  5680601,  9666598,
        8763035,  6453839,  8435504,  2042466,  3940742,  8388472,
        4107856,  4492452,  9382645,  9858567, 10125658, 10523696,
        4111494,  8238289,  2080311,  4049076,  2213917,  1983747,
        8835527, 10282881,  7947331,  6485478,  4142559,  7255713,
        2199463,  6759020,  2226858,  7601292,  8335334,  4555254,
        1988837,  9257922,  7773530,  1961252,  5851105,  7820111,
        4592879, 10724937, 10209997,  5344978,  6787822,  8980396,
        2079377,  5474994,  2121747,  4359962])

In [16]:
def get_random_customer_from_test_data():
    return next(iter(test.take(1)))['customer_id'].numpy()

def display_item_ids(item_ids):
    display(product_data[product_data.main_product_id.isin(item_ids) ])

def display_products_by_customer_id(customer_id):
    display_item_ids(pandas_interaction_data[pandas_interaction_data.customer_id ==  customer_id].main_product_id.tolist())
    

index = tfrs.layers.factorized_top_k.BruteForce(model.user_embedding)
index.index_from_dataset(
    product_ids_dataset.batch(100).map(lambda id: (id, model.product_model(id))))

def predict_user(user_id):
    print('predicting user: ', user_id)
    print('user already bought following products: ')
    display_products_by_customer_id(user_id)
    
    score, predicted_product_ids = index(np.array([user_id]))
    
    print('predicted products: ')
    display_item_ids(predicted_product_ids[0].numpy())
    print('scores: ', score[0].numpy())


user_id = get_random_customer_from_test_data()

predict_user(user_id)

predicting user:  8494801
user already bought following products: 


,main_product_id,productNumber,productName,productColorName,articleNumber,brand__id,mainCategory__id,gender,originCountry,line,...,sizeEu,sizeUk,sizeIndex,shaftLength__value,sole,isTransferee,isSuccessor,duplicate_product__ids,product_sizes,color_mapped
1314,6164348,4064032658598,Sneaker low Materialmix Leder,,76.433.44,6590678,315567,w,SK,H-W,...,40.5,7.0,12.0,0.0,TR,0.0,0.0,"[6164348, 5077410, 6555142, 6446945, 6163947, ...","[35.0, 35.5, 36.0, 37.0, 37.5, 38.0, 38.5, 39....",15


predicted products: 


,main_product_id,productNumber,productName,productColorName,articleNumber,brand__id,mainCategory__id,gender,originCountry,line,...,sizeEu,sizeUk,sizeIndex,shaftLength__value,sole,isTransferee,isSuccessor,duplicate_product__ids,product_sizes,color_mapped
68,3482430,4062862107095,Sneaker low Materialmix Leder/Lederimitat blau,blau,56.966.66,4997827,315567,w,VN,H-W,...,37.5,4.5,7.0,0.0,Gummi-EVA,0.0,0.0,"[3482430, 3482433, 3482437, 3482436, 3482427, ...","[35.0, 35.5, 36.0, 37.0, 37.5, 38.0, 38.5, 39....",1
138,3591304,4062862959946,Sneaker low Rauleder grau,grau,53.420.19,6590677,315567,w,PT,H-W,...,44.0,9.5,17.0,0.0,TR,0.0,0.0,"[3591304, 3591302, 3845754, 3845748, 3845750, ...","[35.0, 35.5, 36.0, 37.0, 37.5, 38.0, 38.5, 39....",4
203,3692992,4062862487708,Sneaker high Rauleder schwarz,schwarz,0908.71.01,3692257,315586,w,PT,H-W,...,36.0,3.5,5.0,11.0,TR,0.0,0.0,"[3692992, 3692995, 3692998, 3788167, 3788154, ...","[35.0, 35.5, 36.0, 37.0, 37.5, 38.0, 38.5, 39....",0
496,3694800,4062862075929,PG0506 Schnürschuh Glattleder braun,braun,0506.50.02,3692257,3692267,m,PT,H-W,...,46.0,11.0,13.0,0.0,EVA,0.0,0.0,"[3694800, 3710999, 3710991, 3711000, 3694802, ...","[39.0, 40.0, 40.5, 41.0, 42.0, 42.5, 43.0, 44....",3
598,4774679,4062862829751,Keilsandalette Rauleder gelb,gelb,62.750.22,6590678,315564,w,PT,F-S,...,35.0,2.5,3.0,0.0,TR,0.0,1.0,"[4774679, 4773177, 4774684, 4773750, 4773755, ...","[35.0, 35.5, 36.0, 37.0, 37.5, 38.0, 38.5, 39....",7
651,4776545,4062862836308,Spangenpumps Lederimitat blau,blau,61.370.46,6590677,315561,w,PT,F-S,...,35.5,3.0,4.0,0.0,EVA,1.0,1.0,"[4776545, 4776453, 4776544, 4776457, 4776443, ...","[35.0, 35.5, 36.0, 37.0, 37.5, 38.0, 38.5, 39....",1
730,4847355,4062862647416,Keilsandalette Glattleder braun,braun,62.043.13,6590678,315564,w,PT,F-S,...,39.0,6.0,10.0,0.0,TR,0.0,0.0,"[4847355, 4777312, 4770237, 4771075, 4847356, ...","[35.0, 35.5, 36.0, 37.0, 37.5, 38.0, 38.5, 39....",3
1365,6132934,4064032998311,Plateau Stiefelette Rauleder grau,grau,74.780.12,6590677,315595,w,SK,H-W,...,35.5,3.0,4.0,14.0,TR,1.0,1.0,"[6132934, 6132933, 6132939, 6132938, 6557051, ...","[35.0, 35.5, 36.0, 37.0, 37.5, 38.0, 38.5, 39....",4
1980,7597174,4064032903506,Riemchensandale Effektleder bronze,bronze,82.711.81,6590678,315564,w,PT,F-S,...,41.0,7.5,13.0,0.0,TR,1.0,1.0,"[7597174, 7597171, 7597178, 7597173, 7597172, ...","[35.0, 36.0, 37.0, 37.5, 38.0, 38.5, 39.0, 40....",12
2304,8631081,4065171236579,Eleganter Pump Rauleder braun,braun,91.441.14,6590677,315593,w,PT,H-W,...,39.0,6.0,10.0,0.0,TPU,0.0,0.0,"[8631081, 8631080, 8631084, 8631079, 8665164, ...","[35.0, 35.5, 37.5, 38.0, 38.5, 39.0, 40.0, 41....",3


scores:  [0.04049084 0.03454798 0.03434926 0.03393251 0.03331247 0.03273907
 0.03255638 0.03013441 0.0283874  0.02823925]


In [17]:
user_id = next(iter(train.take(1)))['customer_id'].numpy()
predict_user(user_id)

predicting user:  11084132
user already bought following products: 


,main_product_id,productNumber,productName,productColorName,articleNumber,brand__id,mainCategory__id,gender,originCountry,line,...,sizeEu,sizeUk,sizeIndex,shaftLength__value,sole,isTransferee,isSuccessor,duplicate_product__ids,product_sizes,color_mapped
862,5150503,4062862543596,Sneaker high Glattleder weiß,weiß,63.400.21,6590677,315586,w,PT,F-S,...,40.5,7.0,12.0,13.0,TR,0.0,0.0,"[5150503, 5150501, 5150502, 5150497, 5150499, ...","[35.0, 35.5, 36.0, 37.0, 37.5, 38.0, 38.5, 39....",2


predicted products: 


,main_product_id,productNumber,productName,productColorName,articleNumber,brand__id,mainCategory__id,gender,originCountry,line,...,sizeEu,sizeUk,sizeIndex,shaftLength__value,sole,isTransferee,isSuccessor,duplicate_product__ids,product_sizes,color_mapped
126,3554637,4062862010111,Plateau Stiefelette Rauleder braun,braun,53.572.18,6590677,5014851,w,VN,H-W,...,39.0,6.0,10.0,15.0,Gummi-EVA,0.0,0.0,"[3554637, 3554643, 3787996, 3787992, 3787995, ...","[35.5, 36.0, 37.0, 37.5, 38.0, 38.5, 39.0, 40....",3
138,3591304,4062862959946,Sneaker low Rauleder grau,grau,53.420.19,6590677,315567,w,PT,H-W,...,44.0,9.5,17.0,0.0,TR,0.0,0.0,"[3591304, 3591302, 3845754, 3845748, 3845750, ...","[35.0, 35.5, 36.0, 37.0, 37.5, 38.0, 38.5, 39....",4
238,3787599,4062862433965,Sneaker high Materialmix Leder multicolour,multicolour,53.591.25,6590677,315586,w,PT,H-W,...,38.0,5.0,8.0,13.0,PU-TPU,0.0,0.0,"[3787599, 3787605, 3787595, 3787607, 3787606, ...","[35.5, 36.0, 37.0, 37.5, 38.0, 38.5, 39.0, 40....",10
601,4772273,4062862780700,Sneaker low Glattleder multicolour,multicolour,63.300.20,6590677,315567,w,PT,F-S,...,40.5,7.0,12.0,0.0,TR,0.0,1.0,"[4772273, 4772276, 4772272, 5184660, 5184669, ...","[35.0, 36.0, 37.0, 37.5, 38.0, 38.5, 39.0, 40....",10
668,4823122,4062862835011,Eleganter Pumps Lederimitat gelb,gelb,61.270.43,6590677,315573,w,PT,F-S,...,39.0,6.0,10.0,0.0,EVA,1.0,1.0,"[4823122, 3473076, 3473083, 3473079, 3473087, ...","[35.5, 36.0, 37.0, 37.5, 38.0, 38.5, 39.0, 40....",7
1501,6570644,4064032577462,Winterstiefelette Lederimitat schwarz,schwarz,71.717.77,6590677,315580,w,PT,H-W,...,43.0,9.0,16.0,24.0,TR,0.0,0.0,"[6570644, 6570641, 6570638, 6570631, 6570642, ...","[35.0, 35.5, 37.5, 38.0, 38.5, 39.0, 40.5, 41....",0
1533,6596615,4064032876923,Hausschuhe Rauleder braun,braun,73.014.14,6590677,8487167,m,PT,H-W,...,42.0,8.0,10.0,0.0,EVA,0.0,0.0,[6596615],[42.0],3
1678,6177675,4064032567845,PG8002 Sneaker low Materialmix Leder/Textil blau,blau,8002.10.02,4997827,3692261,m,CN,H-W,...,44.0,9.5,10.0,0.0,Gummi-EVA,0.0,0.0,"[6177675, 6177671, 6177678, 6177674, 6177679, ...","[39.0, 40.0, 41.0, 42.0, 42.5, 43.0, 44.0, 44....",1
1786,7598061,4064032261569,Riemchensandale Glattleder multicolour,multicolour,82.773.60,6590678,315564,w,PT,F-S,...,40.0,6.5,11.0,0.0,TR,0.0,0.0,"[7598061, 7598058, 7598063, 7598062, 7598057, ...","[35.0, 35.5, 36.0, 37.0, 37.5, 38.0, 38.5, 39....",10
1813,7811882,4064032152836,Plateau Sandale Materialmix Leder weiß,weiß,83.601.21,6590677,315568,w,SK,F-S,...,40.0,6.5,8.0,0.0,TR,0.0,0.0,"[7811882, 7588967, 7811883, 7811880, 7811878, ...","[35.0, 35.5, 36.0, 37.0, 37.5, 38.0, 38.5, 39....",2


scores:  [0.04101894 0.04101    0.0410013  0.03996648 0.03946315 0.03891369
 0.03749887 0.03744262 0.03646318 0.03566917]


In [18]:
#model.save('../../models/gabor/gabor_tensorflow_reco_all_data_3_epochs_v1.h5')